In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import weather_routing
import pandas
import math

In [3]:
year = 2024
##################
min_lat = 21.1
max_lat = 34.1
max_lng = -118
min_lng = -158
# Waypoints for the TransPac 
waypoints = pandas.DataFrame([
    (33.69853, -118.25085, 'Pt. Fermin Bouy'),
    (33.48177, -118.64101, 'Catalina West End'),
    (21.24296, -157.81775, 'Diamond Head Bouy'),
            ], columns=['lat', 'lng', 'name'])
# July 1st, 12:00 PST
Start_Date = f"{year}0701"
Start_Time = 19 #19:00 UTC
##################
#grib_files_dir = "historical_gribs/2024"
wind_data_dir = f"historical_wind_data/{year}"

In [4]:
waypoints

,lat,lng,name
0,33.69853,-118.25085,Pt. Fermin Bouy
1,33.48177,-118.64101,Catalina West End
2,21.24296,-157.81775,Diamond Head Bouy


In [5]:
(FCdate, FCtime,_) = weather_routing.get_grib_time(Start_Date, Start_Time)

In [6]:
# rhumb_route, rhumb_route_t = weather_routing.simulate_shortest_path(
#         waypoints.iloc[0]['lat'], waypoints.iloc[0]['lng'],
#         waypoints.iloc[1]['lat'], waypoints.iloc[1]['lng'],
#         simulation_time=0,
#         FCdate=FCdate,
#         FCtime=FCtime,
#         wind_data_dir=wind_data_dir
#         )
# print(f"Rhumb route takes {rhumb_route_t} steps")

In [7]:
def plot_isochrons_routes(isochrons, rhumb_route=None, min_route=None, saved_isochrons=None):
    import plotly.graph_objects as go
    import plotly.express as px
    # Create the figure
    fig = go.Figure()
    
    for t,isochron in enumerate(isochrons):
        isochron_lats = []
        isochron_lngs = []
        for route in isochron:
            isochron_lats.append(route[-1]['lat'])
            isochron_lngs.append(route[-1]['lng'])
            if t>0:
                # # Add the first route (red line)
                fig.add_trace(go.Scattermapbox(
                    lat=[route[-2]['lat'], route[-1]['lat']],
                    lon=[route[-2]['lng'], route[-1]['lng']],
                    mode='lines',
                    line=dict(color='rgba(255, 0, 0, 0.25)', width=2),
                    hovertext=f"{route[-1]['date']}"
                ))
        fig.add_trace(go.Scattermapbox(
            lat=isochron_lats,
            lon=isochron_lngs,
            mode='lines',
            line=dict(color='rgba(255, 165, 0, 0.75)', width=2),
            hovertext=f"isochron={t+1}"
        ))

    if saved_isochrons is not None:
        for t,isochron in enumerate(saved_isochrons):
            isochron_lats = []
            isochron_lngs = []
            for route in isochron:
                isochron_lats.append(route[-1]['lat'])
                isochron_lngs.append(route[-1]['lng'])
                if t>0:
                    # # Add the first route (red line)
                    fig.add_trace(go.Scattermapbox(
                        lat=[route[-2]['lat'], route[-1]['lat']],
                        lon=[route[-2]['lng'], route[-1]['lng']],
                        mode='lines',
                        line=dict(color='rgba(0, 0, 0, 0.25)', width=2),
                        hovertext=f"{route[-1]['date']}"
                    ))
            fig.add_trace(go.Scattermapbox(
                lat=isochron_lats,
                lon=isochron_lngs,
                mode='lines',
                line=dict(color='rgba(0, 0, 0, 0.75)', width=2),
                hovertext=f"isochron={t+1}"
            ))
    
    if rhumb_route is not None:
        fig.add_trace(go.Scattermapbox(
            lat=rhumb_route['lat'],
            lon=rhumb_route['lng'],
            mode='lines',
            line=dict(color='blue', width=2),
            hovertext=rhumb_route['date']
        ))
    
    if min_route is not None:
        min_route_dt = pandas.DataFrame(min_route)
        fig.add_trace(go.Scattermapbox(
            lat=min_route_dt['lat'],
            lon=min_route_dt['lng'],
            mode='lines',
            line=dict(color='black', width=2),
            hovertext=min_route_dt['date']
        ))
    
    
    # Add scatter points for waypoints
    scatter_points = px.scatter_mapbox(waypoints, 
                                       lat="lat", 
                                       lon="lng", 
                                       hover_name="name")
    fig.add_traces(scatter_points.data)
    # Add the shore boundary line to the figure
    for shore_boundary in weather_routing.shore_boundaries:
        boundary_lat, boundary_lon = zip(*shore_boundary)
        fig.add_trace(go.Scattermapbox(
            lat=boundary_lat,
            lon=boundary_lon,
            mode='lines',
            line=dict(color='green', width=2),
            #name='Boundary'  # Legend label
        ))
    # Update layout for the map
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=9,
        mapbox_center={"lat": 33.7, "lon": -118.5},
        margin={"r":0, "t":0, "l":0, "b":0},
       # width=800,
        height=600,
        showlegend=False
    )
    # Show the plot
    fig.show()

In [16]:

time_step_size = 1 #hour
isochrons=[[
    [{  #start
        'lat': waypoints.iloc[0]['lat'],
        'lng': waypoints.iloc[0]['lng'],
        'date': weather_routing.FCdatetime_to_localtime(FCdate, FCtime,0),
        'dtw' : weather_routing.haversine_distance(
                    waypoints.iloc[0]['lat'],waypoints.iloc[0]['lng'],
                    waypoints.iloc[1]['lat'],waypoints.iloc[1]['lng']), #Distance to Waypoint
        'tdt' : 0, #total distance travled
    }]
]]

simulation_time = 0
for wp_ndx in range(1,len(waypoints)):
    print(f"========== routing to waypoint {wp_ndx} ===========")
    lat_start = waypoints.iloc[wp_ndx-1]['lat']
    lng_start = waypoints.iloc[wp_ndx-1]['lng']
    lat_end = waypoints.iloc[wp_ndx]['lat']
    lng_end = waypoints.iloc[wp_ndx]['lng']

    found_wp = False
    ###
    while not found_wp:
        simulation_time += 1
        print(f"Calculating simulation_time={simulation_time}")
        convex_hull = weather_routing.take_isochron_step(
                isochrons[simulation_time-1], simulation_time-1, FCdate, FCtime, 
                wind_data_dir,lat_start,lng_start,lat_end, lng_end, time_step_size) 
        isochrons.append(convex_hull)
        ####
        min_route = None
        for route in convex_hull:
            #print(route[-1])
            if min_route is None or route[-1]['dtw'] < min_route[-1]['dtw']:
                min_route = route
        print(f"  dtw={min_route[-1]['dtw']:.3f} sog={min_route[-1]['sog']:.3f}", end='')
        if min_route[-1]['dtw'] < min_route[-1]['sog']:
            found_wp = True
            print(" === found waypoint",end='')
            ###
            # add a partial step to the min_route
            min_route.append(
                {
                    'lat':lat_end,
                    'lng':lng_end,
                    'mag':min_route[-1]['mag'],
                    'sog':min_route[-1]['sog'],
                    'dtw':0,
                    'twa':0,
                    'tdt':min_route[-1]['tdt']+min_route[-1]['dtw'],
                    'date':weather_routing.FCdatetime_to_localtime(FCdate, FCtime,simulation_time),
                }
            )
            isochrons.append([min_route])
            ###
        print()
    


========== routing to waypoint 1 ===========
Calculating simulation_time=1
  dtw=19.762 sog=4.680
Calculating simulation_time=2
  dtw=16.299 sog=4.680
Calculating simulation_time=3
  dtw=12.767 sog=4.680
Calculating simulation_time=4
  dtw=8.852 sog=5.620
Calculating simulation_time=5
  dtw=4.925 sog=4.680
Calculating simulation_time=6
  dtw=0.937 sog=4.680 === found waypoint
========== routing to waypoint 2 ===========
Calculating simulation_time=7
  dtw=2195.970 sog=4.680
Calculating simulation_time=8
  dtw=2195.528 sog=4.680
Calculating simulation_time=9
  dtw=2191.739 sog=5.620
Calculating simulation_time=10
  dtw=2190.328 sog=6.590
Calculating simulation_time=11
  dtw=2186.793 sog=5.610
Calculating simulation_time=12
  dtw=2185.147 sog=6.590
Calculating simulation_time=13
  dtw=2182.193 sog=5.660
Calculating simulation_time=14
  dtw=2180.603 sog=5.660
Calculating simulation_time=15
  dtw=2177.455 sog=5.660
Calculating simulation_time=16
  dtw=2175.634 sog=5.660
Calculating simulat

Exception: could not find index for lat or long

In [ ]:
plot_isochrons_routes(isochrons)
#plot_isochrons_routes(isochrons, rhumb_route=rhumb_route, saved_isochrons=saved_nc_isochrons)
#plot_isochrons_routes(isochrons, min_route=min_route)


In [ ]:
len(convex_hull)

In [ ]:
#isochrons

In [ ]:
# min_dtg = 30
# min_route = None
# for route in isochrons[5]:
#     #print(route[-1]['dtw'])
#     if route[-1]['dtw'] < min_dtg:
#         min_dtg = route[-1]['dtw']
#         min_route = route
    

In [ ]:
#len(min_route)

In [ ]:
#len(rhumb_route)

In [ ]:
#min_route